# TGFB1 Part 1 - Importing the data

In this notebook, we take the data from the matrix, gene and cell files and create an AnnData matrix we can use for future analyses.

# Before this notebook was executed, we have aligned our data using these steps:

https://docs.google.com/document/d/1HI_tOIbaw0x1jzzINAAwBlj__7eSiFfDaIO43Hux2D4/edit?usp=sharing

In [1]:
from collections import defaultdict

import numpy as np
import pandas
import scanpy
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

/home/ubuntu/.local/lib/python3.6/site-packages/numba/errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


scanpy==1.4 anndata==0.6.19 numpy==1.17.0 scipy==1.2.1 pandas==0.25.0 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


# Loading genes

Create a function to load all the gene data from the dimension 1 file, creating a list mapping row IDs to genes.

The gene file is simply a list of genes with each line containing the gene name for the appropriate row in the count matrix.

In [2]:
def load_genes(batch, variant):
    genes_result = []
    genes_path = "source/{}.{}.genes.tsv".format(batch, variant)

    with open(genes_path, "r") as genes:
        for gene in genes:
            gene = gene.strip()
            genes_result.append(gene)
    
    return genes_result

In [3]:
genes1 = set(load_genes("control", "all"))
genes1

{'AL391987.3',
 'SLC25A53',
 'SOGA1',
 'RIPOR2',
 'AC018946.1',
 'SLC16A3',
 'PRAMEF4',
 'AC068389.4',
 'HSP90AB1',
 'POLR3D',
 'ZNF236-DT',
 'LINC02137',
 'SEC23B',
 'Z98200.1',
 'TSPO',
 'LIPC-AS1',
 'ACTR6',
 'NDUFB3',
 'ETNK2',
 'AL391261.2',
 'CDK4',
 'TMEM98',
 'MTFR2',
 'AC125793.1',
 'AL157359.2',
 'NCKIPSD',
 'INSIG1',
 'TRPM2',
 'JRK',
 'AC016831.5',
 'CMTM8',
 'PDC',
 'PAX9',
 'EPHA5-AS1',
 'LINC02136',
 'CFHR5',
 'AC009121.2',
 'LINC01133',
 'FAH',
 'LRRN3',
 'AC004076.1',
 'LINC01134',
 'CLEC18A',
 'G3BP1',
 'CCDC96',
 'CCDC39',
 'P2RX6',
 'LINC00115',
 'AC104461.1',
 'RBBP4',
 'ESRP2',
 'EIF3K',
 'KRBOX4',
 'SPDYC',
 'LINC02262',
 'TYROBP',
 'WASHC4',
 'SHH',
 'MARCH11',
 'NOTCH1',
 'PDSS2',
 'AL512274.1',
 'RDH10-AS1',
 'TRIM61',
 'ST5',
 'DBF4',
 'LINC02454',
 'FBXL3',
 'AC008735.3',
 'C18orf65',
 'GTSF1',
 'SPG11',
 'AL359378.1',
 'FGD2',
 'SLC6A17',
 'SSC5D',
 'LY6E-DT',
 'AC129807.1',
 'LMF1-AS1',
 'AC099518.6',
 'LASP1',
 'AL137802.2',
 'POP7',
 'IGSF11',
 'ZC3H7B',

In [4]:
genes2 = set(load_genes("treated", "all"))
genes2

{'AL391987.3',
 'SLC25A53',
 'SOGA1',
 'RIPOR2',
 'SLC16A3',
 'AC068389.4',
 'HSP90AB1',
 'POLR3D',
 'ZNF236-DT',
 'LINC02137',
 'SEC23B',
 'Z98200.1',
 'TSPO',
 'LIPC-AS1',
 'ACTR6',
 'NDUFB3',
 'ETNK2',
 'AL391261.2',
 'CDK4',
 'TMEM98',
 'MTFR2',
 'AC125793.1',
 'AL157359.2',
 'NCKIPSD',
 'INSIG1',
 'TRPM2',
 'JRK',
 'AC016831.5',
 'CMTM8',
 'PDC',
 'PAX9',
 'EPHA5-AS1',
 'LINC02136',
 'CFHR5',
 'AC009121.2',
 'LINC01133',
 'FAH',
 'LRRN3',
 'AC004076.1',
 'LINC01134',
 'CLEC18A',
 'G3BP1',
 'CCDC96',
 'CCDC39',
 'HDGFL1',
 'P2RX6',
 'LINC00115',
 'AC104461.1',
 'RBBP4',
 'ESRP2',
 'EIF3K',
 'KRBOX4',
 'SPDYC',
 'LINC02262',
 'TYROBP',
 'WASHC4',
 'SHH',
 'MARCH11',
 'NOTCH1',
 'PDSS2',
 'AL512274.1',
 'RDH10-AS1',
 'TRIM61',
 'ST5',
 'DBF4',
 'LINC02454',
 'FBXL3',
 'AC008735.3',
 'C18orf65',
 'GTSF1',
 'SPG11',
 'AL359378.1',
 'FGD2',
 'LINC02538',
 'SLC6A17',
 'SSC5D',
 'LY6E-DT',
 'AC129807.1',
 'LMF1-AS1',
 'AC099518.6',
 'LASP1',
 'AL137802.2',
 'LDLRAD4-AS1',
 'POP7',
 'IGSF1

In [5]:
print(len(genes1), len(genes2), len(genes1) + len(genes2))

26740 26590 53330


In [6]:
symdiff = genes1.symmetric_difference(genes2)
symdiff

{'AL023581.1',
 'AC018946.1',
 'PRAMEF4',
 'AC018511.1',
 'AL391421.1',
 'AL021920.3',
 'LINC00557',
 'HDGFL1',
 'GKN1',
 'AC008619.1',
 'AC067852.2',
 'AP003721.4',
 'LINC02538',
 'AL136373.1',
 'LDLRAD4-AS1',
 'PCDHA12',
 'AC005387.2',
 'OR52N5',
 'CHRM2',
 'B3GALT5-AS1',
 'AC073530.1',
 'AC113391.1',
 'HCG24',
 'SST',
 'AL354928.1',
 'LINC01602',
 'AP003500.2',
 'LINC01937',
 'AL136099.1',
 'AL354864.1',
 'AC007347.1',
 'DBNDD2',
 'AC103879.1',
 'AC087667.1',
 'AP002884.2',
 'AC025871.1',
 'MT1HL1',
 'KAAG1',
 'TCEAL3-AS1',
 'LINC01537',
 'LINC02294',
 'AL357033.1',
 'AC007666.1',
 'LHX3',
 'LINC01046',
 'CSHL1',
 'UTS2R',
 'PSG2',
 'LINC00279',
 'AC090510.2',
 'NPY4R2',
 'AL161733.1',
 'ODF3',
 'AL161646.2',
 'AL035443.1',
 'FETUB',
 'ACP4',
 'AC079466.2',
 'AC023932.1',
 'AL499616.1',
 'AC073896.5',
 'ALPP',
 'AC015771.1',
 'AL645937.2',
 'AC027018.1',
 'F9',
 'ANKRD33',
 'SP5',
 'AC068946.2',
 'AC107982.2',
 'LINC01665',
 'LINC01346',
 'AL022238.4',
 'AC093627.7',
 'AL590705.2',


In [7]:
len(symdiff)

2266

# Loading barcodes

Create a function to load barcode data from the dimension 2 file, mapping column IDs to cell barcodes.

The file is simply a list of CBs, with each line corresponding to the same column in the count matrix.

In [8]:
def load_barcodes(batch, variant):
    result = []
    barcode_path = "source/{}.{}.cells.tsv".format(batch, variant)
    
    with open(barcode_path, "r") as barcodes:       
        for barcode in barcodes:
            barcode = barcode.strip()
            barcode += "-" + batch
            result.append(barcode)

    return result

All CBs should be unique.

In [9]:
cells_control = load_barcodes("control", "all")
assert len(cells_control) == len(set(cells_control))

In [10]:
cells_control = load_barcodes("treated", "all")
assert len(cells_control) == len(set(cells_control))

# Loading transcript count matrix

The count matrix is of the format:

    %
    no_genes no_cells no_counts
    19 1 1
    34 1 1
    [...]
    33663 12318 1
    33665 12318 35
    33678 12318 1

where the first number represents the gene, the second number represents the cell, and the third number represents the count.

We then get a `cells` by `genes` count matrix indexed by the cell IDs and gene IDs as described above.

In [11]:
def load_transcript_matrix(batch, variant, genes, barcodes):
    matrix_path = "source/{}.{}.mm".format(batch, variant)
    
    matrix = None
    
    with open(matrix_path, "r") as f:
        first_line = True
        
        for line in f:
            if "%" in line:
                continue
            
            gene, cell, count = [int(i) for i in line.rstrip().split()]
            
            if first_line:
                assert gene == len(genes)
                assert cell == len(barcodes)
                
                first_line = False
                matrix = np.ndarray(shape=(cell, gene), dtype=np.dtype(np.int16))
                continue
            
            
            gene_pos = gene - 1
            cell_pos = cell - 1
            
            matrix[cell_pos, gene_pos] = count
    
    return matrix

# Create Pandas DataFrame from count matrix

Given a count matrix of size `genes x cells`, a list of length `genes` and a list of length `cells`, create a Pandas DataFrame with index `cells` and column labels from `genes`.

In [12]:
def dataframe_from_matrix(m, genes, barcodes):
    assert m.shape == (len(barcodes), len(genes))
    
    # Create a data frame from the count matrix, keyed by the barcode
    # and gene annotations we created.
    result = pandas.DataFrame(m)
    result.index = barcodes
    result.columns = genes
    
    return result

# Construct DataFrame

Given a batch identifier list (i.e. `["S1", "S2"]`) and matrix name list (i.e. `["exon", "intron"]`), create a Pandas DataFrame from the corresponding count matrices and gene/cell dimension file for each. In addition, update a set `genes_set` with all genes that have been encountered in this count matrix, as these are not normalized by default across matrices in a batch (unlike cell IDs).

In [13]:
def construct_dataframes(batches, matrices, genes_set):
    for batch_name in batches:    
        prev_barcodes = None

        for matrix_name in matrices:
            print("Now processing", batch_name, matrix_name)

            genes = load_genes(batch_name, matrix_name)        
            print("Genes list loaded at", len(genes), "items.")

            barcodes = load_barcodes(batch_name, matrix_name)
            print("Barcodes list loaded at", len(barcodes), "items.")

            print("Loading transcription matrix and converting to dataframe...")
            m = load_transcript_matrix(batch_name, matrix_name, genes, barcodes)
            m = dataframe_from_matrix(m, genes, barcodes)

            genes_set.update(m.columns)
            print("Total gene set is now", len(genes_set), "long.")

            if prev_barcodes is not None:
                assert m.index.equals(prev_barcodes)
            else:
                prev_barcodes = m.index
            

            m.to_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))
            print("Count matrix with dimensions", m.shape, "created and pickled for", batch_name, matrix_name)
            print("")

        print("")

# Sorting genes in DataFrames

Given a batch identifier list (i.e. `["S1", "S2"]`) and matrix name list (i.e. `["exon", "intron"]`), load the corresponding DataFrames. Take the `genes_set` generated from the previous process and add to each dataframe the genes that are not encountered in that count matrix, and the counts to 0 for all cells. This is needed as all count matrices need to be the same dimension when the AnnData layers are generated. To normalize the list of genes, we will sort the columns alphabetically as well.

In [14]:
def normalize_dataframe_genes(batches, matrices, genes_set):
    print("Normalizing dimensions of count matrices along genes...")

    for batch_name in batches:    
        for matrix_name in matrices:
            m = pandas.read_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))

            print("Old shape of", batch_name, matrix_name, "was", m.shape)
            missing_genes = genes_set - set(m.columns)
            for missing_gene in missing_genes:
                m[missing_gene] = 0
            print("New shape is", m.shape)

            print("Reordering genes in alphabetical order...")
            print("Old column order was", m.columns)
            m = m.reindex(sorted(m.columns), axis=1)
            print("New column order is", m.columns)

            m.to_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))
            print("Normalized count matrix for", batch_name, matrix_name, "pickled.")
            print("")

        print("")

# Create layered AnnData object from batch matrices

Create a layered AnnData object from frames for each individual batch. In practice, this takes all frames with names in a matrix list (i.e. `["intron", "exon", "spanning"]`) for each batch (i.e. `["S1", "S2"]`), and creates a layered AnnData object with layer names corresponding to `layer_names` (i.e. `["unspliced", "spliced", "ambiguous"]`) from these frames.

Frames should have identical dimensions, indices and column names for this to work.

The layer with name `all` is implicitly used as the main count matrix of the AnnData object.

In [33]:
def create_anndata_from_frames(batches, matrices, layer_names):
    results = []
    
    for batch in batches:
        result = None
        print("Now processing batch", batch, "...")
        
        for matrix, layer in zip(matrices, layer_names):
            print("Loading matrix", matrix, "...")
            m = pandas.read_pickle("/tmp/{}_{}.pickle".format(batch, matrix))
            
            if not result:
                print("Creating AnnData object...")
                
                result = anndata.AnnData(shape=m.shape)
                result.obs = pandas.DataFrame({"cellular_barcode": list(m.index)})
                result.var = pandas.DataFrame({"gene_names": list(m.columns)})
                
            if layer == "all":
                print("Layer is", layer, ", adding it as main count matrix to object...")
                result.X = m
            else:
                print("Adding count matrix as layer", layer, "to AnnData object...")
                result.layers[layer] = m.values
            
            print("")

        result.var_names = [gene for gene in result.var["gene_names"]]
        result.obs_names = [barcode for barcode in result.obs["cellular_barcode"]]
        result.obs_names_make_unique()
        results.append(result)
                
        print("Final AnnData object is", result)
        
        print("Saving layered AnnData object for batch", batch, "...")
        result.write("./write/{}-final.h5ad".format(batch))
        print("")

# Merge AnnData objects along batches

Given a list of AnnData objects corresponding to each batch, create one large AnnData object comprising all batches. Save merged AnnData object for future processing.

In [16]:
def create_merged_anndata(batches):
    result = None
    to_merge = []
    
    for batch in batches:
        if not result:
            print("Merged AnnData file not yet created, making it from batch", batch)
            result = anndata.read_h5ad("./write/{}-final.h5ad".format(batch))
            print("Merged AnnData file is now", result)
        else:
            print("Loading batch", batch)
            curr = anndata.read_h5ad("./write/{}-final.h5ad".format(batch), backed="r+")
            print("Loaded AnnData file is", curr, ", appending to list of objects to concatenate...")
            to_merge.append(curr)
        print("")
    
    print("")        
    print("Concatenating list", to_merge, "to original file")
    result = result.concatenate(*to_merge, batch_categories=batches, index_unique=None)
    print("Concatenated AnnData object is", result, ", saving it to disk...")
    result.write("./write/merged-final.h5ad")

# Run the procedures above
Create the variables for this experiment and run the procedures above, creating an AnnData matrix used in future notebooks.

In [21]:
batches = ('control', 'treated')
matrices = ('exon', 'intron', 'spanning', 'all')
layer_names = ('spliced', 'unspliced', 'ambiguous', 'all')
all_genes = set()

In [18]:
m = construct_dataframes(batches, matrices, all_genes)

Now processing control exon
Genes list loaded at 22488 items.
Barcodes list loaded at 12193 items.
Loading transcription matrix and converting to dataframe...
Total gene set is now 22488 long.
Count matrix with dimensions (12193, 22488) created and pickled for control exon

Now processing control intron
Genes list loaded at 20030 items.
Barcodes list loaded at 12193 items.
Loading transcription matrix and converting to dataframe...
Total gene set is now 26491 long.
Count matrix with dimensions (12193, 20030) created and pickled for control intron

Now processing control spanning
Genes list loaded at 12977 items.
Barcodes list loaded at 12193 items.
Loading transcription matrix and converting to dataframe...
Total gene set is now 26551 long.
Count matrix with dimensions (12193, 12977) created and pickled for control spanning

Now processing control all
Genes list loaded at 26740 items.
Barcodes list loaded at 12193 items.
Loading transcription matrix and converting to dataframe...
Total

In [19]:
normalize_dataframe_genes(batches, matrices, all_genes)

Normalizing dimensions of count matrices along genes...
Old shape of control exon was (12193, 22488)
New shape is (12193, 27798)
Reordering genes in alphabetical order...
Old column order was Index(['WDR13', 'ZDHHC2', 'RPL10A', 'UCHL1', 'RPL13A', 'MICALL2', 'ISCU',
       'TCP11L1', 'RPS18', 'MYL6',
       ...
       'AC011466.3', 'LINC00934', 'AL365204.2', 'AL035425.1', 'OR2G6',
       'AL360014.1', 'LINC02135', 'AC007221.2', 'LILRA1', 'AC083800.1'],
      dtype='object', length=27798)
New column order is Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A4GALT', 'A4GNT', 'AAAS',
       ...
       'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=27798)
Normalized count matrix for control exon pickled.

Old shape of control intron was (12193, 20030)
New shape is (12193, 27798)
Reordering genes in alphabetical order...
Old column order was Index(['PCAT1', 'STK3', 'EXOC4', 'CUX1', 

In [34]:
create_anndata_from_frames(batches, matrices, layer_names)

Now processing batch control ...
Loading matrix exon ...


AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 
AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 


Creating AnnData object...
Adding count matrix as layer spliced to AnnData object...

Loading matrix intron ...
Adding count matrix as layer unspliced to AnnData object...

Loading matrix spanning ...
Adding count matrix as layer ambiguous to AnnData object...

Loading matrix all ...
Layer is all , adding it as main count matrix to object...

Final AnnData object is AnnData object with n_obs × n_vars = 12193 × 27798 
    obs: 'cellular_barcode'
    var: 'gene_names'
    layers: 'spliced', 'unspliced', 'ambiguous'
Saving layered AnnData object for batch control ...

Now processing batch treated ...
Loading matrix exon ...


AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 
AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 


Creating AnnData object...
Adding count matrix as layer spliced to AnnData object...

Loading matrix intron ...
Adding count matrix as layer unspliced to AnnData object...

Loading matrix spanning ...
Adding count matrix as layer ambiguous to AnnData object...

Loading matrix all ...
Layer is all , adding it as main count matrix to object...

Final AnnData object is AnnData object with n_obs × n_vars = 13183 × 27798 
    obs: 'cellular_barcode'
    var: 'gene_names'
    layers: 'spliced', 'unspliced', 'ambiguous'
Saving layered AnnData object for batch treated ...



In [35]:
create_merged_anndata(batches)

Merged AnnData file not yet created, making it from batch control
Merged AnnData file is now AnnData object with n_obs × n_vars = 12193 × 27798 
    obs: 'cellular_barcode'
    var: 'gene_names'
    layers: 'ambiguous', 'spliced', 'unspliced'

Loading batch treated
Loaded AnnData file is AnnData object with n_obs × n_vars = 13183 × 27798 backed at 'write/treated-final.h5ad'
    obs: 'cellular_barcode'
    var: 'gene_names'
    layers: 'ambiguous', 'spliced', 'unspliced' , appending to list of objects to concatenate...


Concatenating list [AnnData object with n_obs × n_vars = 13183 × 27798 backed at 'write/treated-final.h5ad'
    obs: 'cellular_barcode'
    var: 'gene_names'
    layers: 'ambiguous', 'spliced', 'unspliced'] to original file
Concatenated AnnData object is AnnData object with n_obs × n_vars = 25376 × 27798 
    obs: 'batch', 'cellular_barcode'
    var: 'gene_names-control', 'gene_names-treated'
    layers: 'ambiguous', 'spliced', 'unspliced' , saving it to disk...
